In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/test.txt
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/trainval.txt
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_1003.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_593.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_462.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_780.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_1154.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_123.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_389.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_391.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_367.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_106.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_793.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_588.xml
/kaggle/input/iitmhetra/IITM-HeTra_v2/Dataset-1/xmls/frame_160.xml
/kaggle/i

In [2]:
# 1) Remove optional loggers/integrations that can block/crash
!pip -q uninstall -y ray wandb

# 2) Install a newer Ultralytics (fixes some callback issues)
!pip -q install ultralytics==8.3.178 opencv-python pillow

import os, importlib.util
# 3) Belt-and-suspenders: disable wandb via env
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_SILENT"] = "true"

print("ray present?", importlib.util.find_spec("ray"))
print("wandb present?", importlib.util.find_spec("wandb"))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.1 MB/s eta 0:00:00:00:0100:01
ray present? None


In [3]:
from pathlib import Path
import os, shutil, random, json
from PIL import Image
import xml.etree.ElementTree as ET

random.seed(42)
ROOT = Path("/kaggle")
DS   = Path("/kaggle/input/iitmhetra")  # adjust if Kaggle mounts differently
WORK = Path("/kaggle/working")
(WORK / "images" / "all").mkdir(parents=True, exist_ok=True)
(WORK / "labels" / "all").mkdir(parents=True, exist_ok=True)

# Your target classes (index order matters!)
classes = ["car", "bus", "motorcycle", "autorickshaw", "truck"]
class_to_id = {c:i for i,c in enumerate(classes)}
print("Class map:", class_to_id)

Class map: {'car': 0, 'bus': 1, 'motorcycle': 2, 'autorickshaw': 3, 'truck': 4}


In [4]:
!ls -R /kaggle/input/iitmhetra | head -n 120

# Count common annotation types
xmls = list(DS.rglob("*.xml"))
txts = list(DS.rglob("*.txt"))
print(f"Found {len(xmls)} XML files, {len(txts)} YOLO .txt files")

# Also sample a few images to confirm extensions
imgs = list(DS.rglob("*.jpg")) + list(DS.rglob("*.png")) + list(DS.rglob("*.jpeg"))
print("Sample images:", [str(p) for p in imgs[:5]])

/kaggle/input/iitmhetra:
iitm-hetra_v2
IITM-HeTra_v2

/kaggle/input/iitmhetra/iitm-hetra_v2:
IITM-HeTra_v2

/kaggle/input/iitmhetra/iitm-hetra_v2/IITM-HeTra_v2:
Dataset-1
Dataset-2

/kaggle/input/iitmhetra/iitm-hetra_v2/IITM-HeTra_v2/Dataset-1:
images
images_test
test.txt
trainval.txt
xmls
xmls_test

/kaggle/input/iitmhetra/iitm-hetra_v2/IITM-HeTra_v2/Dataset-1/images:
frame_0.jpg
frame_1000.jpg
frame_1001.jpg
frame_1002.jpg
frame_1003.jpg
frame_1004.jpg
frame_1005.jpg
frame_1006.jpg
frame_1007.jpg
frame_1008.jpg
frame_1009.jpg
frame_100.jpg
frame_1010.jpg
frame_1011.jpg
frame_1012.jpg
frame_1013.jpg
frame_1014.jpg
frame_1015.jpg
frame_1016.jpg
frame_1017.jpg
frame_1018.jpg
frame_1019.jpg
frame_101.jpg
frame_1020.jpg
frame_1021.jpg
frame_1022.jpg
frame_1023.jpg
frame_1024.jpg
frame_1025.jpg
frame_1026.jpg
frame_1027.jpg
frame_1028.jpg
frame_1029.jpg
frame_102.jpg
frame_1030.jpg
frame_1031.jpg
frame_1032.jpg
frame_1033.jpg
frame_1034.jpg
frame_1035.jpg
frame_1036.jpg
frame_1037.jpg
fram

In [5]:
# RESET working buckets
import shutil
from pathlib import Path
WORK = Path("/kaggle/working")
for sub in ["images/all", "labels/all", "images/train", "images/val", "images/test",
            "labels/train", "labels/val", "labels/test"]:
    p = WORK / sub
    if p.exists():
        shutil.rmtree(p, ignore_errors=True)
    p.mkdir(parents=True, exist_ok=True)

# Point directly to Dataset-1
DS1 = Path("/kaggle/input/iitmhetra/iitm-hetra_v2/IITM-HeTra_v2/Dataset-1")
IM_DIRS = [DS1/"images"]              # keep it simple; we can add images_test later
XML_DIRS = [DS1/"xmls"]               # matching annotations

# Copy images (no renaming needed since within Dataset-1 there should be no duplicates)
import os
copied = 0
for im_dir in IM_DIRS:
    imgs = list(im_dir.glob("*.jpg")) + list(im_dir.glob("*.jpeg")) + list(im_dir.glob("*.png"))
    for p in imgs:
        dst = WORK/"images"/"all"/p.name
        if not dst.exists():
            try:
                os.link(p, dst)  # fast hardlink
            except Exception:
                shutil.copy2(p, dst)
            copied += 1
print("Images copied to /kaggle/working/images/all:", copied)

Images copied to /kaggle/working/images/all: 1202


In [6]:
import xml.etree.ElementTree as ET
from PIL import Image

classes = ["car","bus","motorcycle","autorickshaw","truck"]
class_to_id = {c:i for i,c in enumerate(classes)}

def voc_to_yolo_line(W,H,x1,y1,x2,y2,cid):
    cx  = ((x1 + x2) / 2.0) / W
    cy  = ((y1 + y2) / 2.0) / H
    ww  = (x2 - x1) / W
    hh  = (y2 - y1) / H
    return f"{cid} {cx:.6f} {cy:.6f} {ww:.6f} {hh:.6f}"

img_map = {p.stem: p for p in (WORK/"images"/"all").glob("*")}

converted = 0
kept = 0
for xml_root in XML_DIRS:
    for x in xml_root.glob("*.xml"):
        stem = x.stem
        if stem not in img_map:
            continue  # only label images we actually copied
        tree = ET.parse(x); root = tree.getroot()
        size = root.find('size'); W = int(size.find('width').text); H = int(size.find('height').text)
        lines = []
        for obj in root.findall('object'):
            name = obj.find('name').text.strip().lower()
            if name not in ("car","bus"):
                continue
            cid = class_to_id[name]
            bnd = obj.find('bndbox')
            x1 = float(bnd.find('xmin').text); y1 = float(bnd.find('ymin').text)
            x2 = float(bnd.find('xmax').text); y2 = float(bnd.find('ymax').text)
            lines.append(voc_to_yolo_line(W,H,x1,y1,x2,y2,cid))
        if lines:
            out = WORK/"labels"/"all"/f"{stem}.txt"
            open(out,"w").write("\n".join(lines)+"\n")
            converted += 1
print("XML→YOLO label files written:", converted)

# sanity: number of images with labels
labeled = len([1 for p in (WORK/'images'/'all').glob('*') if (WORK/'labels'/'all'/f'{p.stem}.txt').exists()])
print("Images that have labels:", labeled)


XML→YOLO label files written: 987
Images that have labels: 987


In [7]:
import random, yaml, shutil
random.seed(42)

images = sorted((WORK/"images"/"all").glob("*"))
labels = {p.stem for p in (WORK/"labels"/"all").glob("*.txt")}
kept_images = [im for im in images if im.stem in labels]

random.shuffle(kept_images)
n = len(kept_images); tr = int(0.7*n); va = int(0.2*n)
splits = {"train": kept_images[:tr], "val": kept_images[tr:tr+va], "test": kept_images[tr+va:]}

for sp, lst in splits.items():
    (WORK/"images"/sp).mkdir(parents=True, exist_ok=True)
    (WORK/"labels"/sp).mkdir(parents=True, exist_ok=True)
    for im in lst:
        lb = WORK/"labels"/"all"/f"{im.stem}.txt"
        shutil.copy2(im, WORK/"images"/sp/im.name)
        shutil.copy2(lb, WORK/"labels"/sp/f"{im.stem}.txt")

print({k: len(v) for k,v in splits.items()})

cfg = {
    "path": str(WORK),
    "train": "images/train",
    "val":   "images/val",
    "test":  "images/test",
    "names": classes,
    "nc": len(classes)
}
with open(WORK/"data.yaml","w") as f:
    yaml.safe_dump(cfg, f, sort_keys=False)
print((WORK/"data.yaml").read_text())


{'train': 690, 'val': 197, 'test': 100}
path: /kaggle/working
train: images/train
val: images/val
test: images/test
names:
- car
- bus
- motorcycle
- autorickshaw
- truck
nc: 5



In [8]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.train(
    data="/kaggle/working/data.yaml",
    epochs=1,
    imgsz=384,
    batch=16,
    workers=0,
    device=0,
    project="runs",
    name="smoke_ok",
    verbose=True
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=384, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=smoke_ok, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=Tru

Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,823 parameters, 3,011,807 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1695.7±802.1 MB/s, size: 100.2 KB)


...

        1/1     0.826G      1.802      2.336      1.258         24        384: 100%|██████████| 44/44 [00:11<00:00,  3.81it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.55it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        197        529     0.0324      0.967      0.563      0.275

1 epochs completed in 0.005 hours.
Optimizer stripped from runs/smoke_ok/weights/last.pt, 6.2MB
Optimizer stripped from runs/smoke_ok/weights/best.pt, 6.2MB

Validating runs/smoke_ok/weights/best.pt...
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  3.86it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        197        529     0.0325      0.967      0.564      0.275
                   car        185        477     0.0584      0.973      0.633      0.282
                   bus         48         52     0.0066      0.962      0.495      0.269
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/smoke_ok


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7932fa68e550>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [9]:
from ultralytics import YOLO
model = YOLO("yolov8s.pt")  # swap to 'yolov8n.pt' if you hit OOM

results = model.train(
    data="/kaggle/working/data.yaml",
    epochs=60,
    imgsz=960,      # use 640 if OOM
    batch=-1,       # auto
    patience=15,
    workers=0,
    device=0,
    project="runs",
    name="traffic_yolov8s",
    cos_lr=True,
    seed=42
)

Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=traffic_yolov8s, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plots=True, pose=12.0, pretrain

train: Scanning /kaggle/working/labels/train.cache... 690 images, 0 backgrounds, 0 corrupt: 100%|██████████| 690/690 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=960 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.74G total, 0.17G reserved, 0.16G allocated, 14.41G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    11137535       64.48         1.177         32.67         94.27        (1, 3, 960, 960)                    list
    11137535         129         1.730         33.72         92.27        (2, 3, 960, 960)                    list
    11137535       257.9         2.907         54.09         113.5        (4, 3, 960, 960)                    list
    11137535       515.8         5.310         83.94         153.6        (8, 3, 960, 960)                    list
    11137535        1032         9.659         173.9         274.9       (16, 3, 960, 960)                    list
AutoBatch: Using batch-size 14 for CUDA:0 8.91G/14.74G (60%) ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1652.2±253.3 MB/s, size: 102.9 KB)


train: Scanning /kaggle/working/labels/train.cache... 690 images, 0 backgrounds, 0 corrupt: 100%|██████████| 690/690 [00:00<?, ?it/s]

...

      60/60      6.98G     0.8503     0.3504      1.063         16        960: 100%|██████████| 50/50 [00:41<00:00,  1.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.23it/s]

                   all        197        529      0.931      0.913       0.96      0.678



60 epochs completed in 0.798 hours.
Optimizer stripped from runs/traffic_yolov8s/weights/last.pt, 22.6MB
Optimizer stripped from runs/traffic_yolov8s/weights/best.pt, 22.6MB

Validating runs/traffic_yolov8s/weights/best.pt...
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.85it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        197        529      0.933      0.929      0.957      0.694
                   car        185        477      0.944       0.96      0.977      0.682
                   bus         48         52      0.921      0.899      0.938      0.705
Speed: 0.3ms preprocess, 7.7ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/traffic_yolov8s


In [10]:
from pathlib import Path
best_path = Path(getattr(model.trainer, "best", "runs/detect/traffic_yolov8s/weights/best.pt"))
print("Best:", best_path)

metrics = model.val(data="/kaggle/working/data.yaml")
print("mAP50-95:", round(metrics.box.map, 4))
print("mAP50   :", round(metrics.box.map50, 4))

import shutil
shutil.copy2(best_path, "/kaggle/working/best.pt")
print("Saved:", "/kaggle/working/best.pt")


Best: runs/traffic_yolov8s/weights/best.pt
Ultralytics 8.3.178 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2060.4±171.2 MB/s, size: 104.8 KB)


val: Scanning /kaggle/working/labels/val.cache... 197 images, 0 backgrounds, 0 corrupt: 100%|██████████| 197/197 [00:00<?, ?it/s]

val: /kaggle/working/images/val/frame_300.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:05<00:00,  2.75it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        197        529      0.933      0.931      0.958      0.695
                   car        185        477      0.944      0.961      0.977      0.681
                   bus         48         52      0.921        0.9      0.939      0.708
Speed: 0.4ms preprocess, 15.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/traffic_yolov8s2
mAP50-95: 0.6947
mAP50   : 0.9578
Saved: /kaggle/working/best.pt
